<a href="https://colab.research.google.com/github/head4ths/CEI/blob/master/scrap_201003_v7_mariaDB_%ED%85%8C%EC%9D%B4%EB%B8%94_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### selenium 설치 : Google Colab 수행용
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver

     |████████████████████████████████| 911kB 2.7MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,338 kB]
Get:14 http://ppa.l

In [ ]:
### selenium 옵션 : 구글 Colab에서 수행용 Driver 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
 
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [ ]:
### BeautifulSoup 설치 
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
# 타이틀 수집 함수
def get_titles(driver,page_no):
    results = []

    driver.get("https://finance.naver.com/item/board.nhn?code=019170&page="+str(page_no))
    req = driver.page_source
    soup=BeautifulSoup(req, 'html.parser')    
    data = soup.select("div.section.inner_sub > table > tbody > tr ")

    for item in data:

      tagA = item.find('a')  
      tagTDs = item.find_all('td')

      # 형식에 안맞으면 Pass
      if tagA is None or len(tagA) <=0:
        continue
      if tagA is not None and len(tagA) > 0:
          if tagA['href'][0:5] != "/item":
            continue

      # 형식에 맞는 건 한 건씩 파싱해서 넣음 
      results.append( (tagTDs[0].text,
                       "신풍제약",
                       tagTDs[1].text.strip().replace('\n', '').replace('\t', ''),
                       "fill content at the next step ",                       
                       tagTDs[3].text,
                       tagTDs[4].text,
                       tagTDs[5].text,
                       tagA.attrs['href']
                       ) )

    print("get_titles done :", time.time() - start)
    return results

In [ ]:
# 컨텐츠 수집함수
def get_contents(driver,df):
  for index, row in df.iterrows():
    #print(index)
    #print("get_contents time :", time.time() - start)    
    try:  
      driver.get("https://finance.naver.com"+row.HREF)
      req = driver.page_source
    except Exception as e:
      print(str(e))
      print(index)      
      print(row)      
    else:
      #print("https://finance.naver.com"+row.HREF)
      soup=BeautifulSoup(req, 'html.parser')    
      data = soup.select("#body")
      content = data[0].text.strip().replace('\n', '').replace('\t', '')
      row.CONTENT = content

  print("get_contents done :", time.time() - start)
  return df

In [ ]:
list1 = []
start = time.time()

for i in range(1,3):  
  list1 +=  get_titles(driver,i) 
  #print(i)
  #print("get_titles time :", time.time() - start)

#print(list1)
df1 = pd.DataFrame(list1, columns=['DATE','ITEM','TITLE','CONTENT','READ','LIKE','DISLIKE','HREF'])
df1 = get_contents(driver, df1)
df1

get_titles done : 1.7338507175445557
get_titles done : 2.28231143951416
get_contents done : 28.230502367019653


,DATE,ITEM,TITLE,CONTENT,READ,LIKE,DISLIKE,HREF
0,2020.10.05 13:43,신풍제약,지금 그래프 십자가?,좋은거에요 나쁜거에요?,28,1,0,/item/board_read.nhn?code=019170&nid=145783219...
1,2020.10.05 13:43,신풍제약,신풍이랑 코로나19 하고 무슨 상관이 ...,병원에서 이미 사용하고 있나요 ?,12,0,0,/item/board_read.nhn?code=019170&nid=145783191...
2,2020.10.05 13:42,신풍제약,용느가 준 기회.[1],저시키가 신풍7에 사서 6에 손절하고 나온 놈이요.저놈덕에 나는 고때 입성했고. 저...,48,1,1,/item/board_read.nhn?code=019170&nid=145783155...
3,2020.10.05 13:42,신풍제약,ㅋㅋㅋ,매출 20억짜리 동전주를12만원에 사면서 가즈아 막 이러네 ㅋㅋㅋ한강 얼기전에 언능...,11,0,1,/item/board_read.nhn?code=019170&nid=145783143...
4,2020.10.05 13:42,신풍제약,신풍 주주들은 어찌 용느님을 찬양하지 않...[1],신풍 주주들은 어찌 용느님을 찬양하지 않으리오?,38,0,2,/item/board_read.nhn?code=019170&nid=145783036...
5,2020.10.05 13:41,신풍제약,외인이던지고 기관이 조금씩 받아먹는 느낌...,그런 느낌 ㅋㅋㅋ,51,0,1,/item/board_read.nhn?code=019170&nid=145783011...
6,2020.10.05 13:41,신풍제약,21층에 사람있는데 인간지표들어와서 못나...,????,45,0,0,/item/board_read.nhn?code=019170&nid=145782973...
7,2020.10.05 13:41,신풍제약,[신풍대한민국] 피라맥스 매출 폭발했습니...[2],거짓말...이러지 마시고~직접 확인해보세요~,366,31,0,/item/board_read.nhn?code=019170&nid=145782882...
8,2020.10.05 13:41,신풍제약,용느 들어와서 못오른다,용느나가면 들어와야함,44,2,2,/item/board_read.nhn?code=019170&nid=145782864...
9,2020.10.05 13:40,신풍제약,그것만 알아두자,지금 주가는고점대비 반토막이야신풍제약 호재 잔뜩 남아서 오르던 내리던 아무 걱정이 없어~,46,2,0,/item/board_read.nhn?code=019170&nid=145782846...
